In [28]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from dotenv import load_dotenv
from typing import TypedDict, Annotated
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.tools import tool
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph

In [2]:
load_dotenv()
llm = ChatOpenAI()


In [5]:
loader = PyPDFLoader("XAI_2.pdf")
docs = loader.load()

In [7]:
splitter  = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
chunks = splitter.split_documents(docs)

In [10]:
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = FAISS.from_documents(chunks, embedding_model)

In [11]:
retriever = vector_store.as_retriever(search_type = 'similarity', search_kwargs = {'k':4})

In [14]:
retriever.invoke("what is the meaning of XAI?")[0]


Document(id='0c03810f-9ebe-4de5-b3ee-9b03805deeaf', metadata={'producer': 'Microsoft® Word LTSC', 'creator': 'Microsoft® Word LTSC', 'creationdate': '2025-03-12T08:31:04+05:30', 'subject': 'World Journal of Advanced Engineering Technology and Sciences, 2025, 14(03), 170-207; DOI: 10.30574/wjaets.2025.14.3.0106', 'author': 'Arunraju Chinnaraju', 'journaltitle': 'World Journal of Advanced Engineering Technology and Sciences', 'keywords': 'Explainable Artificial Intelligence (XAI); Model Interpretability; Decision Transparency; Machine Learning; AI Ethics; Human-Ai Interaction; AI Accountability & Trustworthy', 'doi': '10.30574/wjaets.2025.14.3.0106', 'dcterms.source': 'https://journalwjaets.com/content/explainable-ai-xai-trustworthy-and-transparent-decision-making-theoretical-framework-ai', 'crossmarkdomains': 'wjaets.com', 'issn': '2582-8266', 'moddate': '2025-03-12T14:29:41+05:30', 'title': 'Explainable AI (XAI) for trustworthy and transparent decision-making: A theoretical framework f

In [19]:
@tool
def rag_tool(query):
  """"
    This tool retrive the relavent information from the pdf.
    When the user ask any factal/conceptual question then this answer might be from the doccument.
  """

  result = retriever.invoke(query)

  context = [doc.page_content for doc in result]
  metadata = [doc.metadata for doc in result]

  return {
    'query' : query,
    'context': context,
    'metadata': metadata
  }

In [20]:
tools = [rag_tool]
llm_with_tools = llm.bind_tools(tools)

In [22]:
class chatState(TypedDict):
  messages: Annotated[list[BaseMessage], add_messages]

In [23]:
def chatNode(state: chatState):
  messages = state['messages']
  response = llm_with_tools.invoke(messages)

  return {'messages': [response]}

In [27]:
tool_node = ToolNode(tools)

In [ ]:
graph = StateGraph(chatState)

